# 10A_compare atac to other atac papers

- natgen song data
- duan paper science ASoC
- inoue 2019 cell stem cell

In [1]:
import pandas as pd
import pybedtools
import os, glob
from collections import defaultdict

In [2]:
save_dir = '../data/processed/compare_external/song_atac/'

In [3]:
def get_stats(feature_dict1, feature_dict2, save_prefix, 
              save_bed = False, # save bed A
              save_dir=save_dir):
    stats_dict = {}
    stats_idx = 0
    for my_sampleA, my_bedA in feature_dict1.items():
        num_peaksA = len(my_bedA)
        for my_sampleB, my_bedB in feature_dict2.items():
            num_peaksB = len(my_bedB)
            if save_bed:
                new_name = '__'.join([my_sampleA, my_sampleB,'intersect'])
                my_bedA.intersect(my_bedB,wa=True,f=0.25).saveas(os.path.join(save_dir, new_name+'.bed'))
            stats_dict[stats_idx] = {'sampleA':my_sampleA,
                                    'sampleB':my_sampleB,
                                    'num_peaksA':num_peaksA,
                                    'num_peaksB':num_peaksB}
            stats_dict[stats_idx].update(my_bedA.jaccard(b=my_bedB,f=0.25))
            stats_idx+=1

    stats_df = pd.DataFrame.from_dict(stats_dict,orient='index')
    stats_df['perc_peaksA_captured'] = stats_df.n_intersections/stats_df.num_peaksA
    stats_df['perc_peaksB_captured'] = stats_df.n_intersections/stats_df.num_peaksB
    stats_df.to_csv(os.path.join(save_dir, 'stats_'+save_prefix+'_comp.csv'))   
    return stats_df



In [4]:
my_atac_dir = '../data/interim/merged/atac/'
my_atac_files = glob.glob(os.path.join(my_atac_dir, '*'))
my_atac_files_named = {}
for file in my_atac_files:
    sample = os.path.basename(file).split('_merged')[0]
    df = pybedtools.BedTool(file).sort().merge().to_dataframe()
    df['name']= ['Peak_'+str(i+1) for i in range(len(df))]
    my_atac_files_named[sample] = pybedtools.BedTool.from_dataframe(df)
my_atac_files, my_atac_files_named.keys() 

(['../data/interim/merged/atac/H9_D4_merged.bed',
  '../data/interim/merged/atac/SLC_D0_merged.bed',
  '../data/interim/merged/atac/SL_D0_merged.bed',
  '../data/interim/merged/atac/SLC_D2_merged.bed',
  '../data/interim/merged/atac/SL_D2_merged.bed',
  '../data/interim/merged/atac/H9_D28_merged.bed',
  '../data/interim/merged/atac/H9_D0_merged.bed',
  '../data/interim/merged/atac/Astrocytes_merged.bed',
  '../data/interim/merged/atac/H9_D2_merged.bed',
  '../data/interim/merged/atac/H9_D10_merged.bed'],
 dict_keys(['H9_D4', 'SLC_D0', 'SL_D0', 'SLC_D2', 'SL_D2', 'H9_D28', 'H9_D0', 'Astrocytes', 'H9_D2', 'H9_D10']))

In [5]:
# pybedtools.BedTool(my_atac_files[0]).to_dataframe()[:5]

In [6]:
song_files =glob.glob('/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/song_atac/*.narrowPeak')
song_files_named = {}
for file in song_files:
    sample = os.path.basename(file).split('_')[1].split('.')[0]
    df = pybedtools.BedTool(file).sort().merge().to_dataframe()
    df['name']= ['Peak_'+str(i+1) for i in range(len(df))]
    song_files_named[sample] = pybedtools.BedTool.from_dataframe(df)
    
    
song_files,song_files_named.keys()

(['/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/song_atac/GSE113480_cortical.atac.seq.narrowPeak',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/song_atac/GSE113480_astrocyte.atac.seq.narrowPeak',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/song_atac/GSE113480_motor.atac.seq.narrowPeak',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/song_atac/GSE113480_hippocampal.atac.seq.narrowPeak'],
 dict_keys(['cortical', 'astrocyte', 'motor', 'hippocampal']))

In [134]:
%%time
stats_df_song_my = get_stats(my_atac_files_named, song_files_named, 
                             save_bed=True, save_dir = '../data/processed/compare_external/song_atac/',
                             save_prefix='my_song')
stats_df_song_my.sort_values('perc_peaksA_captured',ascending=True)


CPU times: user 14.4 s, sys: 648 ms, total: 15 s
Wall time: 24.7 s


,sampleA,sampleB,num_peaksA,num_peaksB,intersection,union,jaccard,n_intersections,perc_peaksA_captured,perc_peaksB_captured
14,SLC_D2,motor,71126,67188,22981342,66510151,0.345531,35378,0.497399,0.526552
13,SLC_D2,astrocyte,71126,83383,22409544,73356396,0.305489,36918,0.519051,0.442752
6,SLC_D0,motor,60721,67188,19379126,58861976,0.329230,32421,0.533934,0.482542
10,SL_D0,motor,57826,67188,18128521,57647452,0.314472,31338,0.541936,0.466423
33,H9_D2,astrocyte,53277,83383,16006995,63312878,0.252824,28978,0.543912,0.347529
9,SL_D0,astrocyte,57826,83383,17469545,64580875,0.270506,31863,0.551015,0.382128
5,SLC_D0,astrocyte,60721,83383,18963832,65551717,0.289296,34014,0.560169,0.407925
34,H9_D2,motor,53277,67188,17275572,55769854,0.309765,30269,0.568144,0.450512
17,SL_D2,astrocyte,58421,83383,20269756,67285666,0.301249,33384,0.571438,0.400369
7,SLC_D0,hippocampal,60721,77918,19151774,53739076,0.356385,35123,0.578433,0.450769


In [126]:
# %%time
# stats_dict = {}
# stats_idx = 0
# for my_sample, my_bed in my_atac_files_named.items():
#     num_my_peaks = len(my_bed)
#     for song_sample, song_bed in song_files_named.items():
#         num_song_peaks = len(song_bed)
#         new_name = '__'.join([my_sample, song_sample,'intersect'])
# #         print(new_name)
#         my_bed.intersect(song_bed,wa=True).saveas(os.path.join(save_dir, new_name+'.bed'))
#         stats_dict[stats_idx] = {'my_sample':my_sample,
#                                 'song_sample':song_sample,
#                                 'num_my_peaks':num_my_peaks,
#                                 'num_song_peaks':num_song_peaks}
#         stats_dict[stats_idx].update(my_bed.jaccard(b=song_bed))
#         stats_idx+=1
        

CPU times: user 14.1 s, sys: 599 ms, total: 14.7 s
Wall time: 24.1 s


In [135]:
%%time
stats_df_my_my = get_stats(my_atac_files_named, my_atac_files_named, 
                             save_bed=False, save_dir = '../data/processed/compare_external/song_atac/',
                             save_prefix='my_my')
stats_df_my_my.sort_values('perc_peaksA_captured',ascending=True)


CPU times: user 19.1 s, sys: 745 ms, total: 19.8 s
Wall time: 26.8 s


,sampleA,sampleB,num_peaksA,num_peaksB,intersection,union,jaccard,n_intersections,perc_peaksA_captured,perc_peaksB_captured
37,SLC_D2,Astrocytes,71126,16321,11111412,47364289,0.234595,13261,0.186444,0.812511
35,SLC_D2,H9_D28,71126,16412,10869534,44688338,0.243230,14502,0.203892,0.883622
47,SL_D2,Astrocytes,58421,16321,10627102,39638081,0.268103,12610,0.215847,0.772624
17,SLC_D0,Astrocytes,60721,16321,10252125,36973185,0.277285,13163,0.216778,0.806507
27,SL_D0,Astrocytes,57826,16321,9690562,35069619,0.276324,12860,0.222391,0.787942
...,...,...,...,...,...,...,...,...,...,...
33,SLC_D2,SLC_D2,71126,71126,43083165,43083165,1.000000,71126,1.000000,1.000000
22,SL_D0,SL_D0,57826,57826,29367645,29367645,1.000000,57826,1.000000,1.000000
11,SLC_D0,SLC_D0,60721,60721,31832774,31832774,1.000000,60721,1.000000,1.000000
88,H9_D2,H9_D2,53277,53277,26637098,26637098,1.000000,53277,1.000000,1.000000


In [136]:
%%time
stats_df_song_song = get_stats(song_files_named, song_files_named, 
                             save_bed=False, save_dir = '../data/processed/compare_external/song_atac/',
                             save_prefix='song_song')
stats_df_song_song.sort_values('perc_peaksA_captured',ascending=True)


CPU times: user 5.91 s, sys: 138 ms, total: 6.05 s
Wall time: 7.91 s


,sampleA,sampleB,num_peaksA,num_peaksB,intersection,union,jaccard,n_intersections,perc_peaksA_captured,perc_peaksB_captured
6,astrocyte,motor,83383,67188,20962695,78128408,0.268311,30690,0.368061,0.456778
1,cortical,astrocyte,89917,83383,23828392,77737755,0.306523,39220,0.436180,0.470360
9,motor,astrocyte,67188,83383,20833933,78257170,0.266224,30215,0.449708,0.362364
4,astrocyte,cortical,83383,89917,23784753,77781394,0.305790,38952,0.467146,0.433200
7,astrocyte,hippocampal,83383,77918,22865053,70875798,0.322607,39033,0.468117,0.500950
14,hippocampal,motor,77918,67188,24067585,63398819,0.379622,37995,0.487628,0.565503
2,cortical,motor,89917,67188,27851977,67439723,0.412991,43869,0.487883,0.652929
13,hippocampal,astrocyte,77918,83383,22985889,70754962,0.324866,39583,0.508008,0.474713
3,cortical,hippocampal,89917,77918,27476333,62465115,0.439867,49376,0.549129,0.633692
11,motor,hippocampal,67188,77918,23857668,63608736,0.375069,37145,0.552852,0.476719


takeaways - astro/"cytes seem similar to astrocyts 

the neuron cells are most similar to atac H9D10 and D28
# to do make plot

# 2. duan stuff

convert hg38 to 19


In [113]:
# sort merge then then convert to hg19
duan_files =glob.glob('/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/*.txt')
duan_files_named = {}
for file in duan_files:
    sample = os.path.basename(file).split('.')[0]
    print(sample)
    df = pd.read_table(file)
    df.columns = ['chr','start','end','name','score','strand','summit']
    print(df.shape, 'before')
    df = pybedtools.BedTool.from_dataframe(df).sort().merge().to_dataframe()
    print(df.shape, 'after')
    df['name']= ['Peak_'+str(i+1) for i in range(len(df))]
    bed = pybedtools.BedTool.from_dataframe(df)
    bed.saveas('/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/'+sample+'_hg38.bed')
    duan_files_named[sample] = bed
    
    
duan_files,duan_files_named.keys()

iN_GABA
(278416, 7) before
(278416, 3) after
iN_Dopa
(281210, 7) before
(281210, 3) after
iPSC
(336835, 7) before
(336835, 3) after
NPC
(256085, 7) before
(256085, 3) after
iN_Glut
(256947, 7) before
(256947, 3) after


(['/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iN_GABA.txt',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iN_Dopa.txt',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iPSC.txt',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/NPC.txt',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iN_Glut.txt'],
 dict_keys(['iN_GABA', 'iN_Dopa', 'iPSC', 'NPC', 'iN_Glut']))

In [121]:
# sort merge then then convert to hg19
duan_files =glob.glob('/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/*_hg19.bed')
duan_files_named = {}
for file in duan_files:
    sample = os.path.basename(file).split('.')[0].split('_hg')[0]
    print(sample)
    df = pybedtools.BedTool(file).sort().merge().to_dataframe()
    print(df.shape, 'hg19')
    df['name']= ['Peak_'+str(i+1) for i in range(len(df))]
    duan_files_named[sample] = pybedtools.BedTool.from_dataframe(df)
    
    
duan_files,duan_files_named.keys()

iN_GABA
(275871, 3) hg19
iPSC
(333661, 3) hg19
NPC
(243785, 3) hg19
iN_Glut
(244688, 3) hg19
iN_Dopa
(270214, 3) hg19


(['/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iN_GABA_hg19.bed',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iPSC_hg19.bed',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/NPC_hg19.bed',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iN_Glut_hg19.bed',
  '/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/aay3983_datas1_duan_allele_specific/iN_Dopa_hg19.bed'],
 dict_keys(['iN_GABA', 'iPSC', 'NPC', 'iN_Glut', 'iN_Dopa']))

In [122]:
%%time
stats_df_duan = get_stats(duan_files_named, duan_files_named, save_prefix='duan_duan')
stats_df_duan.sort_values('perc_peaksA_captured',ascending=True)


CPU times: user 29.6 s, sys: 282 ms, total: 29.8 s
Wall time: 39.2 s


,sampleA,sampleB,num_peaksA,num_peaksB,intersection,union,jaccard,n_intersections,perc_peaksA_captured,perc_peaksB_captured
7,iPSC,NPC,333661,243785,57411506,270152593,0.212515,128052,0.383779,0.525266
8,iPSC,iN_Glut,333661,244688,61796763,275991652,0.223908,130329,0.390603,0.532633
5,iPSC,iN_GABA,333661,275871,79251011,336802747,0.235304,136729,0.409784,0.495627
9,iPSC,iN_Dopa,333661,270214,85434277,316337318,0.270073,145080,0.434813,0.536908
2,iN_GABA,NPC,275871,243785,64478325,215892268,0.298660,132602,0.480667,0.543930
1,iN_GABA,iPSC,275871,333661,79251011,336802747,0.235304,136729,0.495627,0.409784
22,iN_Dopa,NPC,270214,243785,66465147,199623283,0.332953,136313,0.504463,0.559153
11,NPC,iPSC,243785,333661,57411506,270152593,0.212515,128052,0.525266,0.383779
3,iN_GABA,iN_Glut,275871,244688,75388458,215206451,0.350308,145856,0.528711,0.596090
16,iN_Glut,iPSC,244688,333661,61796763,275991652,0.223908,130329,0.532633,0.390603


In [129]:
%%time
stats_df_duan_my = get_stats(my_atac_files_named, duan_files_named, 
                             save_bed=True, save_dir = '../data/processed/compare_external//duan_atac/',
                             save_prefix='my_duan')
stats_df_duan_my.sort_values('perc_peaksA_captured',ascending=True)


CPU times: user 53.6 s, sys: 947 ms, total: 54.6 s
Wall time: 1min 22s


,sampleA,sampleB,num_peaksA,num_peaksB,intersection,union,jaccard,n_intersections,perc_peaksA_captured,perc_peaksB_captured
34,H9_D0,iN_Dopa,45669,270214,16698727,177120958,0.094279,33325,0.729707,0.123328
44,H9_D2,iN_Dopa,53277,270214,19228569,177556492,0.108295,39205,0.735871,0.145089
33,H9_D0,iN_Glut,45669,244688,16546064,113290441,0.146050,33851,0.741225,0.138344
32,H9_D0,NPC,45669,243785,16100240,103511949,0.155540,34046,0.745495,0.139656
42,H9_D2,NPC,53277,243785,18268224,104309341,0.175135,39963,0.750099,0.163927
40,H9_D2,iN_GABA,53277,275871,20122186,190945038,0.105382,40664,0.763256,0.147402
30,H9_D0,iN_GABA,45669,275871,17712435,190389413,0.093033,34943,0.765136,0.126664
43,H9_D2,iN_Glut,53277,244688,19367372,113434509,0.170736,41304,0.775269,0.168803
12,SL_D0,NPC,57826,243785,20818784,104489328,0.199243,45108,0.780064,0.185032
14,SL_D0,iN_Dopa,57826,270214,22137557,177378051,0.124804,45439,0.785788,0.168159


# 3. inoue 2019 cell stem cell

['/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/inoue_atac/GSM3163878_ATACseq_7TP_24hr_rep1_peaks.narrowPeak.gz',


In [143]:
inoue_files = glob.glob('/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/external_data/inoue_atac/*')
inoue_rep_bed_dict = defaultdict(list)
for file in inoue_files:
    _, _, _, time, rep, _ = os.path.basename(file).split('_')
    print(time, rep)
    inoue_rep_bed_dict[time].append(pybedtools.BedTool(file).sort().merge())
    
    
    
    

24hr rep1
6hr rep1
48hr rep2
3hr rep1
0hr rep1
24hr rep2
0hr rep2
72hr rep2
3hr rep2
3hr rep1
0hr rep1
72hr rep1
72hr rep2
3hr rep2
0hr rep2
12hr rep2
6hr rep2
48hr rep1
72hr rep1
12hr rep1


In [146]:
inoue_files_named = {}
for time, bed_arr in inoue_rep_bed_dict.items():
    print(time)
    bed_merge = bed_arr[0] + bed_arr[1]
    df = bed_merge.to_dataframe()
    print(df.shape, 'hg19')
    df['name']= ['Peak_'+str(i+1) for i in range(len(df))]
    inoue_files_named[time] = pybedtools.BedTool.from_dataframe(df)
    

inoue_files_named.keys()

24hr
(25339, 3) hg19
6hr
(25310, 3) hg19
48hr
(32867, 3) hg19
3hr
(37397, 3) hg19
0hr
(17706, 3) hg19
72hr
(30846, 3) hg19
12hr
(26692, 3) hg19


dict_keys(['24hr', '6hr', '48hr', '3hr', '0hr', '72hr', '12hr'])

In [148]:
%%time
stats_df_inoue = get_stats(inoue_files_named, 
                          inoue_files_named, save_prefix='inoue_inoue')
stats_df_inoue.sort_values('perc_peaksA_captured',ascending=True)


CPU times: user 5.53 s, sys: 331 ms, total: 5.86 s
Wall time: 7.91 s


,sampleA,sampleB,num_peaksA,num_peaksB,intersection,union,jaccard,n_intersections,perc_peaksA_captured,perc_peaksB_captured
18,48hr,0hr,32867,17706,3747344,19400976,0.193152,12282,0.373688,0.693663
25,3hr,0hr,37397,17706,4073177,13933252,0.292335,14969,0.400273,0.845420
39,72hr,0hr,30846,17706,3278906,10351801,0.316747,13066,0.423588,0.737942
4,24hr,0hr,25339,17706,3529146,15306727,0.230562,11720,0.462528,0.661923
46,12hr,0hr,26692,17706,3775520,15608364,0.241891,12589,0.471639,0.711002
11,6hr,0hr,25310,17706,3695952,14525939,0.254438,12377,0.489016,0.699029
21,3hr,24hr,37397,25339,6954532,19841484,0.350505,18494,0.494532,0.729863
26,3hr,72hr,37397,30846,5199600,16391250,0.317218,18812,0.503035,0.609868
35,72hr,24hr,30846,25339,5012483,17407811,0.287944,17030,0.552098,0.672087
27,3hr,12hr,37397,26692,7801160,19542867,0.399182,21051,0.562906,0.788663


In [149]:
%%time
stats_df_inoue_my = get_stats(my_atac_files_named, inoue_files_named, 
                             save_bed=True, save_dir = '../data/processed/compare_external/inoue_atac/',
                             save_prefix='my_inoue')
stats_df_inoue_my.sort_values('perc_peaksA_captured',ascending=True)


CPU times: user 9.63 s, sys: 1.03 s, total: 10.7 s
Wall time: 20.3 s


,sampleA,sampleB,num_peaksA,num_peaksB,intersection,union,jaccard,n_intersections,perc_peaksA_captured,perc_peaksB_captured
25,SLC_D2,0hr,71126,17706,3254055,44852253,0.072551,9853,0.138529,0.556478
32,SL_D2,0hr,58421,17706,3263295,36632495,0.089082,9911,0.169648,0.559754
11,SLC_D0,0hr,60721,17706,3560418,33295499,0.106934,11264,0.185504,0.636169
18,SL_D0,0hr,57826,17706,3638937,30751851,0.118332,11664,0.201709,0.658760
60,H9_D2,0hr,53277,17706,3744595,27915646,0.134140,12029,0.225782,0.679374
...,...,...,...,...,...,...,...,...,...,...
51,Astrocytes,48hr,16321,32867,7349463,26168250,0.280854,10763,0.659457,0.327471
41,H9_D28,12hr,16412,26692,6825055,20010393,0.341076,11415,0.695528,0.427656
36,H9_D28,6hr,16412,25310,6841133,18832322,0.363266,11606,0.707165,0.458554
38,H9_D28,3hr,16412,37397,5445408,20012585,0.272099,11744,0.715574,0.314036
